In [1]:
%load_ext autoreload
%load_ext autotime

time: 408 µs (started: 2023-03-06 17:38:25 -08:00)


In [2]:
import sys
sys.path.append("../")
import warnings
warnings.filterwarnings('ignore')

time: 858 µs (started: 2023-03-06 17:38:25 -08:00)


Some logs are stored in VAST with some stored in GPFS.

Set `log_dir` to `/vast1/` when running the notebook on Quartz, otherwise use `/gpfs1/`.

In [3]:
log_dir = "/p/vast1/iopp/recorder_app_logs/cm1/nodes-32/workflow-4/_parquet"
# log_dir = "/p/gpfs1/iopp/recorder_app_logs/hacc|/nodes-32/workflow-0/_parquet"
#log_dir = "/p/gpfs1/iopp/recorder_app_logs/montage/nodes-32/_parquet"
#log_dir = "/p/gpfs1/iopp/recorder_app_logs/montage_pegasus/nodes-32/_parquet"
#log_dir = "/p/gpfs1/iopp/recorder_app_logs/lbann-cosmoflow/nodes-32/_parquet" # FIXME
#log_dir = "/p/gpfs1/iopp/recorder_app_logs/mummi-wemul/nodes-32-ppn-32/_parquet"# FIXME
# log_dir = "/p/gpfs1/iopp/recorder_app_logs/lbann-jag/nodes-32/_parquet"
#log_dir = "/p/gpfs1/iopp/recorder_app_logs/genome_pegasus/nodes-32/_parquet"


time: 870 µs (started: 2023-03-06 17:38:25 -08:00)


`RecorderAnalyzer` is the main part that initializes the cluster. It creates `LocalCluster` or `LSFCluster` depending on the `force_local` variable. When it is set to `true`, it will just create a `LocalCluster`. 

In [4]:
%autoreload 2

from wisio.recorder import VIEW_TYPES, RecorderAnalyzer

recorder_analyzer = RecorderAnalyzer(
    cluster_manager_args=dict(
        force_local=True,
        cluster_settings=dict(
            cores=32,
            dashboard_port=3646,
            local_directory="/var/tmp/dask-recorder",
            log_file="%J.log",
            worker_queue="pdebug"
        )
    ),
    working_dir='.recorder',
    debug=True
)

recorder_analyzer


time: 7.05 s (started: 2023-03-06 17:38:25 -08:00)


In [5]:
[client.dashboard_link for client in recorder_analyzer.cluster_manager.clients]


['http://192.168.131.2:3646/status']

time: 3.38 ms (started: 2023-03-06 17:38:33 -08:00)


From this point on, every step is exactly the same within the `analyze_parquet` method of `RecorderAnalyzer`.

In [6]:
%autoreload 2

global_min_max = recorder_analyzer.load_global_min_max(log_dir=log_dir)
global_min_max


{'file_id': [901974072163427748, 4531734248727221027],
 'proc_id': [6860131432589748884, 6860131432673569428],
 'tmid': [0, 6683055996]}

time: 14.9 ms (started: 2023-03-06 17:38:42 -08:00)


In [9]:
%autoreload 2

from wisio._recorder.analysis import (
    compute_main_view,
    compute_max_io_time,
    compute_view
)

time: 2.27 ms (started: 2023-03-06 17:38:53 -08:00)


`compute_main_view` automatically does `persist` before returning the DataFrame. You may play with that part if the `main_view` doesn't fit within the allocated memory. That said, `persist`ing it increases performance significantly.

In [10]:
%autoreload 2

main_view = compute_main_view(
    log_dir=log_dir,
    global_min_max=global_min_max,
    view_types=VIEW_TYPES,
)

main_view


,trange,file_id,proc_id,io_cat,acc_pat,duration_sum,index_count,size_min,size_max,size_sum,file_name,proc_name,func_id,read_time,write_time,metadata_time,read_size,write_size,metadata_size,read_count,write_count,metadata_count,data_count,data_size,data_time,sequential_time,random_time,sequential_size,random_size,sequential_count,random_count
npartitions=1,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
,int64,int64,int64,int64,int64,float32,int64,int64,int64,int64,object,object,object,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


time: 1.48 s (started: 2023-03-06 17:38:54 -08:00)


`max_io_time` is just a `dd.Scalar` since I tried to compute everyting lazily.

In [11]:
%autoreload 2

max_io_time = compute_max_io_time(main_view=main_view)
max_io_time

dd.Scalar<series-..., dtype=float32>

time: 29.6 ms (started: 2023-03-06 17:39:03 -08:00)


You may configure `delta` and `cut` using the following variables.

In [12]:
DELTA = 0.0001
CUT = 0.5

time: 961 µs (started: 2023-03-06 17:39:04 -08:00)


The following is how all the views (including `expanded_view` and `bottleneck_view`-I didn't like the `cut_view` name). So, if you want to play with that part `compute_view` is the method you should look for.

In [13]:
%autoreload 2

import itertools as it

def _view_permutations(r: int):
    return it.permutations(VIEW_TYPES, r + 1)

views = {}
for view_permutation in it.chain.from_iterable(map(_view_permutations, range(len(VIEW_TYPES)))):
    # Compute view
    views[view_permutation] = compute_view(
        main_view=main_view,
        views=views,
        view_permutation=view_permutation,
        max_io_time=max_io_time,
        delta=DELTA,
        cut=CUT
    )


len(views)

15

time: 1.58 s (started: 2023-03-06 17:39:10 -08:00)


In [14]:
views[('trange',)]['expanded_view'].dtypes

trange                int64
file_id               int64
proc_id               int64
io_cat                int64
acc_pat               int64
duration_sum        float32
index_count           int64
size_min              int64
size_max              int64
size_sum              int64
file_name            object
proc_name            object
func_id              object
read_time             int64
write_time            int64
metadata_time         int64
read_size             int64
write_size            int64
metadata_size         int64
read_count            int64
write_count           int64
metadata_count        int64
data_count            int64
data_size             int64
data_time             int64
sequential_time       int64
random_time           int64
sequential_size       int64
random_size           int64
sequential_count      int64
random_count          int64
duration_score         int8
duration_cut        float16
duration_pero       float32
duration_perr       float32
dtype: object

time: 7.86 ms (started: 2023-03-06 17:39:14 -08:00)


In [15]:
views[('trange',)]['expanded_view'].head()

,trange,file_id,proc_id,io_cat,acc_pat,duration_sum,index_count,size_min,size_max,size_sum,...,sequential_time,random_time,sequential_size,random_size,sequential_count,random_count,duration_score,duration_cut,duration_pero,duration_perr
0,1,901974072163427748,6860131432590862996,3,0,0.000003,1,0,0,0,...,0.00000,0,0,0,0,0,1,0.0,6.804209e-07,2.883792e-08
1,1,4531734244992817467,6860131432590862996,4,0,0.001609,5,16777216,16777216,16777216,...,0.00147,0,16777216,0,1,0,1,0.0,3.909990e-04,1.657151e-05
2,3,901974072163427748,6860131432589814420,3,0,0.000003,1,0,0,0,...,0.00000,0,0,0,0,0,1,0.0,7.533231e-07,3.192770e-08
3,3,901974072163427748,6860131432589879956,3,0,0.000003,1,0,0,0,...,0.00000,0,0,0,0,0,1,0.0,7.533231e-07,3.192770e-08
4,3,901974072163427748,6860131432589945492,3,0,0.000002,1,0,0,0,...,0.00000,0,0,0,0,0,1,0.0,4.617142e-07,1.956859e-08


time: 651 ms (started: 2023-03-06 17:39:15 -08:00)


In [16]:
views[('file_id',)]['expanded_view'].head()

,trange,file_id,proc_id,io_cat,acc_pat,duration_sum,index_count,size_min,size_max,size_sum,...,sequential_time,random_time,sequential_size,random_size,sequential_count,random_count,duration_score,duration_cut,duration_pero,duration_perr
0,1,4531734244992817467,6860131432590862996,4,0,0.001609,5,16777216,16777216,16777216,...,0.001470,0,16777216,0,1,0,1,0.0,0.000391,1.657322e-05
1,3,4531734244992817467,6860131432589748884,4,0,0.000034,5,16777216,16777216,16777216,...,0.000009,0,16777216,0,1,0,1,0.0,0.000008,3.491809e-07
2,3,4531734244992817467,6860131432589814420,4,0,0.000037,5,16777216,16777216,16777216,...,0.000009,0,16777216,0,1,0,1,0.0,0.000009,3.831720e-07
3,3,4531734244992817467,6860131432589879956,4,0,0.000037,5,16777216,16777216,16777216,...,0.000010,0,16777216,0,1,0,1,0.0,0.000009,3.769918e-07
4,3,4531734244992817467,6860131432589945492,4,0,0.000041,5,16777216,16777216,16777216,...,0.000012,0,16777216,0,1,0,1,0.0,0.000010,4.223132e-07


time: 286 ms (started: 2023-03-06 17:39:17 -08:00)


In [17]:
views[('proc_id',)]['expanded_view'].head()

,trange,file_id,proc_id,io_cat,acc_pat,duration_sum,index_count,size_min,size_max,size_sum,...,sequential_time,random_time,sequential_size,random_size,sequential_count,random_count,duration_score,duration_cut,duration_pero,duration_perr
0,1,901974072163427748,6860131432590862996,3,0,0.000003,1,0,0,0,...,0.00000,0,0,0,0,0,1,0.0,6.804209e-07,2.885244e-08
1,1,4531734244992817467,6860131432590862996,4,0,0.001609,5,16777216,16777216,16777216,...,0.00147,0,16777216,0,1,0,1,0.0,3.909990e-04,1.657985e-05
2,3,901974072163427748,6860131432592370324,3,0,0.000003,1,0,0,0,...,0.00000,0,0,0,0,0,5,0.5,7.533231e-07,3.194377e-08
3,3,4531734244992817467,6860131432592370324,4,0,0.000038,5,16777216,16777216,16777216,...,0.00001,0,16777216,0,1,0,5,0.5,9.258584e-06,3.925993e-07
4,4,4531734247156679037,6860131432592370324,3,0,0.000003,1,0,0,0,...,0.00000,0,0,0,0,0,5,0.5,7.533231e-07,3.194377e-08


time: 252 ms (started: 2023-03-06 17:39:19 -08:00)


The following just initializes the bottleneck detector.

In [18]:
%autoreload 2

from wisio._recorder.bottlenecks import RecorderBottleneckDetector

bottleneck_detector = RecorderBottleneckDetector(
    logger=recorder_analyzer.logger,
    log_dir=log_dir,
)

bottleneck_detector


time: 6.14 ms (started: 2023-03-06 17:39:21 -08:00)


This part is where our nested loops lie. The output type is `{view_type: bottleneck_dict}`.

In [19]:
%autoreload 2

bottlenecks = bottleneck_detector.detect_bottlenecks(
    views=views,
    view_types=VIEW_TYPES,
)

len(bottlenecks)

15

time: 25.3 s (started: 2023-03-06 17:39:23 -08:00)


In [20]:
bottlenecks[('trange',)]

{5: {'llc': {'duration_sum': 2.1918428,
   'index_count': 32,
   'size_min': 0,
   'size_max': 0,
   'size_sum': 0,
   'func_id': ['open'],
   'read_time': 0.0,
   'write_time': 0.0,
   'metadata_time': 2.191842794418335,
   'read_size': 0,
   'write_size': 0,
   'metadata_size': 0,
   'read_count': 0,
   'write_count': 0,
   'metadata_count': 32,
   'data_count': 0,
   'data_size': 0,
   'data_time': 0.0,
   'sequential_time': 0.0,
   'random_time': 0,
   'sequential_size': 0,
   'random_size': 0,
   'sequential_count': 0,
   'random_count': 0,
   'duration_score': 5,
   'duration_cut': 0.5,
   'duration_pero': 0.53263414,
   'duration_perr': 0.022574356},
  'proc_name': {'0': {'llc': {'duration_sum': 1.970968,
     'index_count': 1,
     'size_min': 0,
     'size_max': 0,
     'size_sum': 0,
     'func_id': ['open'],
     'read_time': 0.0,
     'write_time': 0.0,
     'metadata_time': 1.9709680080413818,
     'read_size': 0,
     'write_size': 0,
     'metadata_size': 0,
     'read_c

time: 91.6 ms (started: 2023-03-06 17:40:00 -08:00)


The following saves the bottlenecks per `view_type`. 

In [21]:
%autoreload 2

recorder_analyzer.save_bottlenecks(log_dir=log_dir, bottlenecks=bottlenecks)


time: 6.59 s (started: 2023-03-06 17:40:40 -08:00)
